In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import albumentations as A
from albumentations.tensorflow import ToTensorV2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

In [ ]:
train_dir = "dataset/train"
val_dir = "dataset/val"
test_dir = "dataset/test"
img_size = (224, 224)
batch_size = 32
num_classes = 5

In [ ]:
train_transform = A.Compose([
    A.Resize(img_size, img_size),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.4),
    A.Blur(blur_limit=3, p=0.3),
    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, fill_value=0, p=0.4),
    A.Normalize(),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(img_size, img_size),
    A.Normalize(),
    ToTensorV2()
])

In [ ]:
class RiceLeafDataset(tf.keras.utils.Sequence):
    def __init__(self, directory, transform=None, batch_size=32, shuffle=True):
        self.directory = directory
        self.transform = transform
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.classes = sorted(os.listdir(directory))
        self.image_paths = []
        self.labels = []
        for idx, cls in enumerate(self.classes):
            cls_folder = os.path.join(directory, cls)
            for img_name in os.listdir(cls_folder):
                self.image_paths.append(os.path.join(cls_folder, img_name))
                self.labels.append(idx)
        self.indices = np.arange(len(self.image_paths))
        if shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, index):
        batch_indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        images, labels = [], []
        for i in batch_indices:
            image = cv2.imread(self.image_paths[i])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            label = self.labels[i]
            if self.transform:
                image = self.transform(image=image)["image"]
            images.append(image)
            labels.append(label)
        images = np.stack(images)
        labels = tf.keras.utils.to_categorical(labels, num_classes=len(self.classes))
        return images, labels

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

In [ ]:
train_gen = RiceLeafDataset(train_dir, transform=train_transform, batch_size=batch_size)
val_gen = RiceLeafDataset(val_dir, transform=val_transform, batch_size=batch_size, shuffle=False)
test_gen = RiceLeafDataset(test_dir, transform=val_transform, batch_size=batch_size, shuffle=False)


In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:150]:  # Keep lower layers frozen
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

In [ ]:
model.save("efficientnet_rice_leaf_albu.h5")

In [ ]:
y_true = []
y_pred = []

for images, labels in test_gen:
    preds = model.predict(images)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(np.argmax(labels, axis=1))

class_labels = test_gen.classes

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()